In [ ]:
from pathlib import Path

import torch

from src.dataset import COCODataset, COCODataloader
from src.models.unet import UNet
from src.trainer import Trainer
from src.utils import (
    load_config,
    seed_everything,
    load_weights,
    download_coco_data
)
from src.visuals import Segmentor

print("Pytorch version:", torch.__version__)
print("CUDA enabled:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name())

In [ ]:
CONFIG = load_config()
seed_everything(CONFIG.seed)
model = load_weights(
    model=UNet(),
    filepath=Path(r".runs/v1/weights_best.pth")
)
segmentor = Segmentor(model)
CONFIG

In [ ]:
data = download_coco_data(config=CONFIG, split="train", max_samples=CONFIG.num_samples)
train_end = int(CONFIG.num_samples * 0.8)
val_start, val_end = train_end, train_end + int(CONFIG.num_samples * 0.1)
test_start = val_end
print(data)

# import fiftyone
# fiftyone.launch_app(data)

In [ ]:
train_dataset = COCODataset(data[:train_end], summarize=False)
val_dataset = COCODataset(data[val_start:val_end], summarize=False)
test_dataset = COCODataset(data[test_start:], summarize=False)
train_dl = COCODataloader(train_dataset, batch_size=CONFIG.batch_size, pin_memory=True)
val_dl = COCODataloader(val_dataset, batch_size=CONFIG.batch_size, pin_memory=True)
test_dl = COCODataloader(test_dataset, batch_size=CONFIG.batch_size, pin_memory=True)
trainer = Trainer(model, dataloaders=(train_dl, val_dl))

In [ ]:
for dataloader in (train_dl, val_dl, test_dl):
    _, metrics = trainer.eval(dataloader)
    trainer.log_metrics(metrics)

In [ ]:
for i in range(5):
    image, gt_mask = test_dataset[i]
    segmentor.segment(image)
    segmentor.visualize_mask(image, gt_mask)

In [ ]:
segmentor.segment_url("https://www.lrt.lt/img/2020/05/05/648522-374636-756x425.jpg")